In [1]:
import pandas as pd
import json
import csv

In [2]:
customers=pd.read_csv("customers.csv")
accounts=pd.read_csv("accounts.csv")
branches=pd.read_csv("branches.csv")
transactions=pd.read_csv("transactions.csv")
credit_cards=pd.read_json("credit_cards.json")
support_tickets=pd.read_csv("support_tickets.csv")
loans=pd.read_json("loans.json")

In [3]:
import mysql.connector

In [4]:
conn=mysql.connector.connect(
    host='localhost',
    user='root',
    password='M@h!UV127'
)


In [5]:
cursor=conn.cursor()

In [6]:
cursor.execute("CREATE DATABASE IF NOT EXISTS customer_data")
print("Database created or alread exists")

Database created or alread exists


In [7]:
cursor.execute("USE customer_data;")

In [8]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS accounts (
        customer_id VARCHAR(50) PRIMARY KEY,
        account_balance FLOAT,
        last_updated DATETIME
    );
""")

In [9]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS branches (
        Branch_ID INT PRIMARY KEY,
        Branch_Name VARCHAR(100),
        City VARCHAR(50),
        Manager_Name VARCHAR(50),
        Total_Employees INT,
        Branch_Revenue FLOAT,
        Opening_Date DATE,
        Performance_Rating INT
    );
""")

In [10]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS customers (
        customer_id VARCHAR(10) PRIMARY KEY,
        name VARCHAR(50),
        gender CHAR(1),
        age INT,
        city VARCHAR(50),
        account_type VARCHAR(50),
        join_date DATE
    );
""")

In [11]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS transactions (
        txn_id VARCHAR(10) PRIMARY KEY,
        customer_id CHAR(5),
        txn_type VARCHAR(20),
        amount FLOAT,
        txn_time DATETIME,
        status VARCHAR(20)
    );
""")

In [12]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS credit_cards (
        Card_ID INT,
        Customer_ID INT,
        Account_ID INT,
        Branch VARCHAR(200),
        Card_Number DECIMAL(40,0),
        Card_Type VARCHAR(20),
        Card_Network VARCHAR(20),
        Credit_Limit INT,
        Current_Balance FLOAT,
        Issued_Date DATE,
        Expiry_Date DATE,
        Status VARCHAR(15)   
    );
""")
conn.commit()

In [13]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS loans (
        Loan_ID INT PRIMARY KEY,
        Customer_ID INT,
        Account_ID INT,
        Branch VARCHAR(200),
        Loan_Type VARCHAR(20),
        Loan_Amount INT,
        Interest_Rate FLOAT,
        Loan_Term_Months INT,
        Start_Date DATE,
        End_Date DATE,
        Loan_Status VARCHAR(25)   
    );
""")
conn.commit()

In [14]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS support_tickets (
        Ticket_ID VARCHAR(10),
        Customer_ID VARCHAR(15),
        Account_ID VARCHAR(15),
        Loan_ID VARCHAR(10),
        Branch_Name VARCHAR(200),
        Issue_Category VARCHAR(50),
        Description VARCHAR(500),
        Date_Opened DATE,
        Date_Closed DATETIME,
        Priority VARCHAR(50),
        Status VARCHAR(50),
        Resolution_Remarks VARCHAR(500),
        Support_Agent VARCHAR(200),
        Channel VARCHAR(50),            
        Customer_Rating INT             
    );
""")
conn.commit()

In [15]:
cursor.execute("SELECT * from branches")
pd.DataFrame(cursor.fetchall())

,0,1,2,3,4,5,6,7
0,1,"Wang, Mcdonald and Church Branch",Kellitown,Kevin Jones,141,16849300.0,2008-08-01,2
1,2,Crawford-Torres Branch,Angelaburgh,Andrea Steele,39,5822280.0,2015-09-17,1
2,3,Bridges Inc Branch,Johnsonfort,Michael Coleman,195,2114590.0,2012-07-30,3
3,4,Garcia Group Branch,New Kyleberg,Dawn Cooper,144,3390390.0,2018-06-27,2
4,5,"Arroyo, Moore and Freeman Branch",West Coryborough,Samantha Kennedy,42,2771910.0,2012-01-04,5
...,...,...,...,...,...,...,...,...
515,516,Lowery and Sons Branch,Lake Charles,Daniel Hunt,130,6562890.0,2007-06-22,4
516,517,Brooks Group Branch,New Christina,Rodney Browning,19,19078800.0,2021-07-20,1
517,518,Harris-Marshall Branch,Emilymouth,Lorraine Johnson,177,2801630.0,2011-08-18,5
518,519,"Meadows, Taylor and Butler Branch",Port Michael,David Smith,76,17653100.0,2006-05-21,4


In [34]:
cursor.execute("SELECT txn_type, SUM(amount) AS 'amount' FROM transactions GROUP BY txn_type")
df=pd.DataFrame(cursor.fetchall())
df.columns=['txn_type','amount']
df


,txn_type,amount
0,deposit,1.975748e+08
1,withdrawal,1.513873e+08
2,transfer,5.059280e+07
3,online fraud,2.382950e+07
4,purchase,7.383403e+07


In [35]:
cursor.execute("SELECT * from accounts")
pd.DataFrame(cursor.fetchall())

,0,1,2
0,C0001,293730.0,2025-05-24 09:46:29
1,C0002,150735.0,2025-05-24 09:46:29
2,C0003,170156.0,2025-05-24 09:46:29
3,C0004,311384.0,2025-05-24 09:46:29
4,C0005,60320.8,2025-05-24 09:46:29
...,...,...,...
495,C0496,149136.0,2025-05-24 09:46:29
496,C0497,376153.0,2025-05-24 09:46:29
497,C0498,320254.0,2025-05-24 09:46:29
498,C0499,484758.0,2025-05-24 09:46:29


In [36]:
# Q1: How many customers exist per city, and what is their average account balance?
cursor.execute("SELECT c.city, COUNT(c.customer_id), ROUND(AVG(a.account_balance),2) as 'avg_balance' FROM customers AS c LEFT JOIN accounts AS a ON a.customer_id=c.customer_id GROUP BY city")
df1=pd.DataFrame(cursor.fetchall())
df1.columns=['city','customer_id','avg_balance']
df1

,city,customer_id,avg_balance
0,Kimberlyburgh,1,293729.88
1,Port David,3,209838.99
2,West Laurieton,1,170156.20
3,Amyville,1,311383.97
4,New Craigport,1,60320.75
...,...,...,...
482,Mullenland,1,149135.80
483,Shermanfurt,1,376153.38
484,Nataliemouth,1,320254.16
485,Millerstad,1,484758.19


In [37]:
# Q2: Which account type (Savings, Current, Loan, etc.) holds the highest total balance?
cursor.execute("SELECT c.account_type, ROUND(SUM(account_balance),2) AS 'total_balance' FROM customers AS c LEFT JOIN accounts AS a ON c.customer_id=a.customer_id GROUP BY account_type ORDER BY total_balance DESC LIMIT 1")
df2=pd.DataFrame(cursor.fetchall())
df2.columns=['account_type','total_balance']
df2

,account_type,total_balance
0,Savings,61880483.1


In [20]:
# Q3: Who are the top 10 customers by total account balance across all account types?
cursor.execute("SELECT c.name, SUM(a.account_balance) AS 'total_balance' FROM customers AS c LEFT JOIN accounts AS a ON c.customer_id=a.customer_id GROUP BY name ORDER BY total_balance DESC LIMIT 10")
df3=pd.DataFrame(cursor.fetchall())
df3.columns=['name','total_balance']
df3


,name,total_balance
0,Anthony Ross,513495.640625
1,Amy Nguyen,498739.500000
2,Carolyn Higgins,498317.406250
3,Lisa Russell,497040.593750
4,Amber Williams,494779.968750
5,Audrey Peck,494699.906250
6,Alvin Compton,484758.187500
7,Sharon Bennett DVM,482746.375000
8,Denise Campbell,481746.593750
9,Jeffrey Smith,481568.125000


In [21]:
# Q4: Which customers opened accounts in 2023 with a balance above ₹1,00,000?
cursor.execute("SELECT c.name, a.account_balance, c.join_date FROM customers AS c LEFT JOIN accounts AS a ON c.customer_id=a.customer_id WHERE account_balance>100000 AND join_date>'2022-12-31'")
df4=pd.DataFrame(cursor.fetchall())
df4.columns=['name','account_balance','join_date']
df4

,name,account_balance,join_date
0,Richard Wilson,293730.0,2023-08-27
1,Holly Parker,150735.0,2023-12-08
2,Jasmine Morris,291737.0,2023-10-09
3,Carolyn Higgins,498317.0,2024-01-25
4,Jesus Davenport,345239.0,2023-06-07
...,...,...,...
143,Curtis Nash,133229.0,2024-03-24
144,Matthew Willis,210818.0,2024-02-09
145,Rachel Jackson,137153.0,2024-03-31
146,Alvin Compton,484758.0,2023-06-16


In [22]:
# Q5: What is the total transaction volume (sum of amounts) by transaction type?
cursor.execute("SELECT txn_type, SUM(amount) as'amount' FROM transactions GROUP BY txn_type")
df5=pd.DataFrame(cursor.fetchall())
df5.columns=['txn_type','amount']
df5

,txn_type,amount
0,deposit,1.975748e+08
1,withdrawal,1.513873e+08
2,transfer,5.059280e+07
3,online fraud,2.382950e+07
4,purchase,7.383403e+07


In [23]:
# Q6: Which accounts have more than 3 failed transactions in a single month?
cursor.execute("select c.account_type, count(t.status) as 'failed_transactions' from customers as c left join transactions as t on t.customer_id=c.customer_id where t.status='failed' group by c.account_type having count(status)>3 order by 'failed_transactions' desc")
df6=pd.DataFrame(cursor.fetchall())
df6.columns=['account_type','failed_transactions']
df6

,account_type,failed_transactions
0,Savings,747
1,Current,750


In [24]:
# Q7: Which are the top 5 branches by total transaction volume in the last 6 months?

In [25]:
# Q8: Which accounts have 5 or more high-value transactions above ₹2,00,000?
cursor.execute("SELECT c.name, SUM(t.amount) AS 'total' FROM customers AS c LEFT JOIN transactions AS t ON c.customer_id=t.customer_id WHERE amount>200000 GROUP BY name")
(cursor.fetchall())




[]

In [26]:
#Q9: What is the average loan amount and interest rate by loan type (Personal, Auto, Home, etc.)?
cursor.execute("SELECT loan_type, ROUND(AVG(loan_amount),2) AS 'average_amount', ROUND(AVG(interest_rate),2) AS 'average_rate' FROM loans GROUP BY loan_type")
df9=pd.DataFrame(cursor.fetchall())
df9.columns=['loan_type','average_amount','average_rate']
df9

,loan_type,average_amount,average_rate
0,Personal,2461543.75,10.46
1,Business,2793968.87,10.63
2,Auto,2399260.98,10.53
3,Home,2344576.66,10.87
4,Education,2393947.70,10.11


In [27]:
#Q10: Which customers currently hold more than one active or approved loan?
cursor.execute("SELECT customer_id, COUNT(loan_status) AS 'total' FROM loans WHERE loan_status IN ('active', 'approved') GROUP BY customer_id HAVING total>1")
df10=pd.DataFrame(cursor.fetchall())
df10.columns=['customer_id','total']
df10

,customer_id,total
0,1411,3
1,1383,3
2,1073,2
3,1059,2
4,1430,2
5,1292,3
6,1216,2
7,1359,2
8,1231,4
9,1155,2


In [28]:
# Q11: Who are the top 5 customers with the highest outstanding (non-closed) loan amounts?
cursor.execute("SELECT customer_id, loan_amount FROM loans WHERE loan_status='Defaulted' ORDER BY loan_amount DESC LIMIT 5")
df11=pd.DataFrame(cursor.fetchall())
df11.columns=['customer_id','loan_amount']
df11

,customer_id,loan_amount
0,1510,4888307
1,1533,4886771
2,1362,4873847
3,1018,4817920
4,1454,4731455


In [29]:
# Q12: Which branch holds the highest total account balance?
cursor.execute("SELECT b.branch_name FROM branches AS b LEFT JOIN accounts AS a ON b.branch_id=a.customer_id GROUP BY branch_name ORDER BY SUM(account_balance) DESC LIMIT 1")
df12=pd.DataFrame(cursor.fetchall())
df12.columns=['branch_name']
df12

,branch_name
0,"Wang, Mcdonald and Church Branch"


In [30]:
# Q13: What is the branch performance summary showing total customers, total loans, and transaction volume?
cursor.execute("SELECT (SELECT COUNT(DISTINCT customer_id) FROM transactions) AS 'total_customers', (SELECT COUNT(DISTINCT loan_id) FROM loans) AS 'total_loans', (SELECT sum(amount) FROM transactions) AS 'transaction_volume'")
df13=pd.DataFrame(cursor.fetchall())
df13.columns=['total_customers','total_loans','transaction_volume']
df13

,total_customers,total_loans,transaction_volume
0,500,553,4.972185e+08


In [31]:
# Q14: Which issue categories have the longest average resolution time?
cursor.execute("SELECT issue_category, AVG(DATEDIFF(date_closed, date_opened)) AS 'resolution_time' FROM support_tickets GROUP BY issue_category ORDER BY resolution_time DESC LIMIT 1")
df14=pd.DataFrame(cursor.fetchall())
df14.columns=['issue_category', 'average_resolution_time']
df14

,issue_category,average_resolution_time
0,Card Not Working,18.5000


In [38]:
# Q15: Which support agents have resolved the most critical tickets with high customer ratings (≥4)?
cursor.execute("SELECT DISTINCT support_agent FROM support_tickets WHERE priority='Critical' AND customer_rating>=4")
df15=pd.DataFrame(cursor.fetchall())
df15.columns=['support_agent']
df15

,support_agent
0,Udyati Mody
1,Ekanta Devan
2,Rehaan Balakrishnan
3,Gaurika Pillay
4,Gabriel Mutti
5,Owen Mahajan
6,Kalpit Ahuja
7,Advik Bhagat
8,Ranveer Roy
9,Wishi Chhabra


In [39]:
import streamlit as st